In [17]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F

import numpy as np

In [18]:
# Load and normalizde the data

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batchSize = 5
validSize = 0.2 # use 20% of train set as validation

trainValidSet = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
testSet = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

trainSet, validSet = torch.utils.data.random_split(trainValidSet, [int(len(trainValidSet)*(1-validSize)), int(len(trainValidSet)*validSize)])

trainLoader = torch.utils.data.DataLoader(trainSet, batch_size=batchSize, shuffle=True)
validLoader = torch.utils.data.DataLoader(validSet, batch_size=batchSize, shuffle=True)
testLoader = torch.utils.data.DataLoader(testSet, batch_size=batchSize, shuffle=False)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [19]:
# Writer for tensorBoard
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

In [20]:
len(trainLoader)*batchSize, len(validLoader)*batchSize, len(testLoader)*batchSize, next(iter(testLoader))[0][0].shape

(40000, 10000, 10000, torch.Size([3, 32, 32]))

In [21]:
# Define the network class

class ConvNet(nn.Module):
    def __init__(self, activation):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 100)
        self.fc2 = nn.Linear(100, 10)
        self.activation = activation

    def forward(self, x):
        x = self.pool(self.activation(self.conv1(x)))
        x = self.pool(self.activation(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = self.activation(self.fc1(x))
        x = self.fc2(x)
        return x


In [22]:
def trainNetwork(network, optimizer, lossFunction, epochs, writer):
    network.train()
    for epoch in range(epochs):
        
        ### TRAINING ###
        trainLoss = 0
        correctTrain = 0
        totalTrain = 0
        for batch_nr, (images, labels) in enumerate(trainLoader):

            # Predict
            predictions = network(images)

            # Get loss and backpropogate
            loss = lossFunction(predictions, labels)
            loss.backward() 

            # Optimize parameters (weights and biases) and remove gradients after
            optimizer.step() 
            optimizer.zero_grad()

            # Save loss for whole epoch
            trainLoss += loss.item()
            
            # Calculate training accuracy
            _, predictions = torch.max(predictions, 1) 
            correctTrain += (predictions == labels).sum().item() 
            totalTrain += len(images)
        

        trainLoss /= len(trainLoader)
        trainAccuracy = 100 * correctTrain / totalTrain
        writer.add_scalar(f"Loss/train - {network.activation.__name__}/{type(optimizer).__name__}", trainLoss, epoch)
        writer.add_scalar(f"Accuracy/train - {network.activation.__name__}/{type(optimizer).__name__}", trainAccuracy, epoch)

        ### VALIDATION ###
        validLoss = 0
        correctValid = 0
        totalValid = 0
        for batch_nr, (images, labels) in enumerate(validLoader):

            # Predict
            predictions = network(images)

            # Get loss
            loss = lossFunction(predictions, labels)

            # Save loss for whole epoch
            validLoss += loss.item()

            # Calculate vaildation accuracy
            _, predictions = torch.max(predictions, 1) 
            correctValid += (predictions == labels).sum().item() 
            totalValid += len(images)

        validLoss /= len(validLoader)
        validAccuracy = 100 * correctValid / totalValid
        writer.add_scalar(f"Loss/valid - {network.activation.__name__}/{type(optimizer).__name__}", validLoss, epoch)
        writer.add_scalar(f"Accuracy/valid - {network.activation.__name__}/{type(optimizer).__name__}", validAccuracy, epoch)

        # Print reuslt of epoch
        print(f'Epoch [{epoch+1}/{epochs}] \t Training Loss: {trainLoss} \t Validation Loss: {validLoss} \t Traning Acc: {trainAccuracy}% \t Validation Acc: {validAccuracy}%')

    writer.flush()

In [23]:
def testNetwork(network):
    network.eval()
    correctTest = 0
    totalTest = 0

    ### TESTING ###
    with torch.no_grad(): 
        for batch_nr, (images, labels) in enumerate(testLoader):
            # Get predictions and get the amount of correct predicitons
            predictions = network(images)
            _, predictions = torch.max(predictions, 1) 
            correctTest += (predictions == labels).sum().item() 
            totalTest += len(images)
        
    print(f"Test Accuracy: {100 * correctTest / totalTest}%")

In [24]:
learningRate = 0.0001
epochs = 10

In [25]:
networkLeakyRelu = ConvNet(F.leaky_relu)

optimizerSGD = torch.optim.SGD(networkLeakyRelu.parameters(), lr=learningRate)
lossFunction = nn.CrossEntropyLoss()

In [26]:
trainNetwork(networkLeakyRelu, optimizerSGD, lossFunction, epochs, writer)

Epoch [1/10] 	 Training Loss: 2.301979294896126 	 Validation Loss: 2.3011810780763624 	 Traning Acc: 10.31% 	 Validation Acc: 10.79%
Epoch [2/10] 	 Training Loss: 2.299531454116106 	 Validation Loss: 2.298276985883713 	 Traning Acc: 11.295% 	 Validation Acc: 12.03%
Epoch [3/10] 	 Training Loss: 2.2955668054521086 	 Validation Loss: 2.2932186316251757 	 Traning Acc: 12.515% 	 Validation Acc: 13.27%
Epoch [4/10] 	 Training Loss: 2.2880773162841797 	 Validation Loss: 2.2828538990020752 	 Traning Acc: 14.2825% 	 Validation Acc: 15.83%
Epoch [5/10] 	 Training Loss: 2.2716469617784023 	 Validation Loss: 2.259039041399956 	 Traning Acc: 16.6175% 	 Validation Acc: 18.16%
Epoch [6/10] 	 Training Loss: 2.2349335293620824 	 Validation Loss: 2.208185036122799 	 Traning Acc: 20.0% 	 Validation Acc: 21.75%
Epoch [7/10] 	 Training Loss: 2.169193065032363 	 Validation Loss: 2.1368503108024597 	 Traning Acc: 22.7375% 	 Validation Acc: 24.2%
Epoch [8/10] 	 Training Loss: 2.1060376297086476 	 Validation 

In [27]:
testNetwork(networkLeakyRelu)

Test Accuracy: 28.26%


In [28]:
networkLeakyRelu2 = ConvNet(F.leaky_relu)

optimizerAdam = torch.optim.Adam(networkLeakyRelu2.parameters(), lr=learningRate)
trainNetwork(networkLeakyRelu2, optimizerAdam, lossFunction, epochs, writer)

Epoch [1/10] 	 Training Loss: 1.7730630318447949 	 Validation Loss: 1.626636968910694 	 Traning Acc: 35.9025% 	 Validation Acc: 41.31%
Epoch [2/10] 	 Training Loss: 1.542037549626082 	 Validation Loss: 1.5069540883004666 	 Traning Acc: 44.045% 	 Validation Acc: 46.02%
Epoch [3/10] 	 Training Loss: 1.4399068030230702 	 Validation Loss: 1.4201623430326582 	 Traning Acc: 48.4375% 	 Validation Acc: 50.13%
Epoch [4/10] 	 Training Loss: 1.3681173711270094 	 Validation Loss: 1.370998761795461 	 Traning Acc: 51.5125% 	 Validation Acc: 51.53%
Epoch [5/10] 	 Training Loss: 1.310241325924173 	 Validation Loss: 1.3397283364832402 	 Traning Acc: 53.2825% 	 Validation Acc: 52.88%
Epoch [6/10] 	 Training Loss: 1.262663916900754 	 Validation Loss: 1.2926815413087607 	 Traning Acc: 55.06% 	 Validation Acc: 54.77%
Epoch [7/10] 	 Training Loss: 1.2227582611609251 	 Validation Loss: 1.262059724971652 	 Traning Acc: 56.59% 	 Validation Acc: 55.4%
Epoch [8/10] 	 Training Loss: 1.1860978130698203 	 Validatio

In [29]:
testNetwork(networkLeakyRelu2)

Test Accuracy: 58.79%


In [30]:
networkTanh = ConvNet(torch.tanh)
trainNetwork(networkTanh, optimizerAdam, lossFunction, epochs, writer)

Epoch [1/10] 	 Training Loss: 2.30577752161026 	 Validation Loss: 2.304220145821571 	 Traning Acc: 8.5625% 	 Validation Acc: 9.03%
Epoch [2/10] 	 Training Loss: 2.3057775217592718 	 Validation Loss: 2.304220148563385 	 Traning Acc: 8.5625% 	 Validation Acc: 9.03%
Epoch [3/10] 	 Training Loss: 2.3057775202393533 	 Validation Loss: 2.3042201437950136 	 Traning Acc: 8.5625% 	 Validation Acc: 9.03%
Epoch [4/10] 	 Training Loss: 2.3057775230407715 	 Validation Loss: 2.3042201424837114 	 Traning Acc: 8.5625% 	 Validation Acc: 9.03%
Epoch [5/10] 	 Training Loss: 2.3057775205671787 	 Validation Loss: 2.304220145702362 	 Traning Acc: 8.5625% 	 Validation Acc: 9.03%
Epoch [6/10] 	 Training Loss: 2.305777522474527 	 Validation Loss: 2.304220144867897 	 Traning Acc: 8.5625% 	 Validation Acc: 9.03%
Epoch [7/10] 	 Training Loss: 2.305777522474527 	 Validation Loss: 2.3042201455831526 	 Traning Acc: 8.5625% 	 Validation Acc: 9.03%
Epoch [8/10] 	 Training Loss: 2.3057775197029113 	 Validation Loss: 2.

In [31]:
testNetwork(networkTanh)

Test Accuracy: 8.19%
